In [9]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [10]:
# Load dataset
df = pd.read_csv('C:/Users/Kornelius/Desktop/Data 2/nips-papers/papers.csv', header = 0, sep = ';', error_bad_lines=False)
# Drop the columns not needed
df = df.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

In [11]:
# Drop a row by condition
df = df[df.abstract != 'Abstract Missing']
# Print out the first rows of papers

In [12]:
data = df.abstract.values.tolist()

In [13]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(data):
    stop_free = " ".join([i for i in data.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(data).split() for data in data]  

In [14]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [15]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(16129 unique tokens: ['activity', 'also', 'analysis', 'applied', 'assume']...)


In [16]:
texts = data
# less than 10 documents,
id2word = corpora.Dictionary(texts)
id2word.filter_extremes(no_below = 10)

In [17]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [20]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"algorithm" + 0.015*"model" + 0.011*"learning" + 0.011*"problem" + '
  '0.011*"method" + 0.010*"data" + 0.008*"show" + 0.007*"result" + '
  '0.006*"approach" + 0.006*"function"')]


In [21]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.055*"model" + 0.052*"network" + 0.026*"neural" + 0.012*"neuron" + '
  '0.012*"system" + 0.011*"memory" + 0.011*"deep" + 0.010*"human" + '
  '0.009*"convolutional" + 0.009*"pattern"'),
 (1,
  '0.024*"algorithm" + 0.024*"learning" + 0.015*"problem" + 0.014*"online" + '
  '0.013*"regret" + 0.012*"study" + 0.011*"setting" + 0.011*"bound" + '
  '0.010*"time" + 0.009*"show"'),
 (2,
  '0.022*"data" + 0.020*"learning" + 0.018*"model" + 0.016*"image" + '
  '0.015*"task" + 0.014*"method" + 0.013*"feature" + 0.012*"approach" + '
  '0.011*"label" + 0.010*"training"'),
 (3,
  '0.043*"algorithm" + 0.033*"problem" + 0.031*"method" + 0.021*"function" + '
  '0.013*"approach" + 0.012*"optimization" + 0.011*"learning" + 0.010*"show" + '
  '0.010*"stochastic" + 0.010*"inference"'),
 (4,
  '0.019*"model" + 0.015*"distribution" + 0.015*"data" + 0.012*"matrix" + '
  '0.010*"result" + 0.010*"sample" + 0.010*"bound" + 0.010*"analysis" + '
  '0.009*"graph" + 0.009*"show"')]
[(0,
  '0.100*"matrix" + 0.

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -7.141634942078457

Perplexity5:  -7.074515027500307

Perplexity10:  -7.084844565504205

Perplexity20:  -7.0840042105178

Perplexity50:  -7.054904438093043

Perplexity100:  -18.260099025758166

Coherence Score:  0.2357683522987768

Coherence Score5:  0.42141946621857673

Coherence Score10:  0.3998864985796514

Coherence Score20:  0.39464949528233373

Coherence Score50:  0.3813321304434668

Coherence Score100:  0.4109786328060066

Coherence ScoreUMass:  -1.4635242209070833

Coherence Score5UMass:  -1.9889367592225617

Coherence Score10UMass:  -2.6230649866647338

Coherence Score20UMass:  -5.725642045561722

Coherence Score50UMass:  -8.333768014296751

Coherence Score100UMass:  -9.330269234163294
